# HDB Resale Price Predictor & Visualisation

This project aims to create a data pipeline with the help of availale APIs (Data.gov.sg and OneMap) to build a web-based application for
1. HDB Price visualisation
2. HDB Price prediction

The prototype aims to read latest data directly from data.gov.sg and perform ETL (Extract, Transform, and Load) to a local/web database of choice.

In [1]:
import os
os.chdir('f:\python_stuff\ml_webapp')
print(f'Working directory: {str(os.getcwd())}')

from modules.utils import *
from etl import *

# Get the correct Logger
etl_logger = logging.getLogger('etl')
etl_logger

Working directory: f:\python_stuff\ml_webapp


<Logger etl (INFO)>

## Data Wrangling Contents
1. API call data
2. Data Wrangling
3. Feature Engineering

## 1. Getting the data through API call

### Wrapper functions
* To time function calls
* To error handle HTTPerrors and other Exceptions
* To cache API calls

In [2]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
    # if config['automation'] & datetime.now().day != 30:
    #     print('Exiting ETL script - script will only run on 30th of each month')
    #     sys.exit()

    # Accounts for filepathing local and in pythonanywhere
    if config['local']:
        cache_filepath = config['local_cache_filepath']
    else:
        os.chdir(config['web_prefix'])
        cache_filepath = 'project_cache'
    
    # files to append to
    output_file_train = config['train']
    output_file_test = config['test']

    # Determines whether to extract all data for current year, or particular year and months
    use_curr_datetime = config['use_datetime']
    if use_curr_datetime:
        timestamp = datetime.now()
        years = [timestamp.year]
        months = [x for x in range(1, timestamp.month+1)]
    else:
        years = config['years']
        months = config['months']

etl_logger.info(f"{'-'*50}New run started {'-'*50}")
etl_logger.info(f'Data extraction settings:')
etl_logger.info(f'\tuse_curr_datetime: {use_curr_datetime}')
etl_logger.info(f'\tyear(s): {years}')
etl_logger.info(f'\tmonth(s): {months}')

# Enable caching
session = requests_cache.CachedSession(cache_filepath, backend="sqlite")

### Details for Data.gov.sg API call can be found at
https://data.gov.sg/dataset/ckan-datastore-search

In [3]:
# There is now a limit to the API calls, so split to individual call for each month instead
df = pd.DataFrame()
etl_logger.info('Making API calls to data.gov.sg')
for year in years:
    for month in months:
        temp_df = datagovsg_api_call_v2(year=year, month=month)
        etl_logger.info(f'\tData df shape received: {temp_df.shape}')
        if df.empty:
            df = temp_df
        else:
            df = pd.concat([df, temp_df])
etl_logger.info('\t\tCompleted')

In [4]:
df

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,143397,2023-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44,Improved,1979,55 years 05 months,267000
1,143398,2023-01,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,04 TO 06,49,Improved,1977,53 years 06 months,300000
2,143399,2023-01,ANG MO KIO,2 ROOM,314,ANG MO KIO AVE 3,04 TO 06,44,Improved,1978,54 years 01 month,280000
3,143400,2023-01,ANG MO KIO,2 ROOM,314,ANG MO KIO AVE 3,07 TO 09,44,Improved,1978,54 years 01 month,282000
4,143401,2023-01,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,01 TO 03,45,Improved,1986,62 years 01 month,289800
...,...,...,...,...,...,...,...,...,...,...,...,...
1938,171109,2024-01,YISHUN,5 ROOM,820,YISHUN ST 81,04 TO 06,121,Improved,1988,63 years 08 months,650000
1939,171110,2024-01,YISHUN,EXECUTIVE,391,YISHUN AVE 6,01 TO 03,146,Maisonette,1988,63 years 05 months,850000
1940,171111,2024-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,04 TO 06,146,Maisonette,1988,63 years 11 months,783000
1941,171112,2024-01,YISHUN,EXECUTIVE,356,YISHUN RING RD,04 TO 06,146,Maisonette,1988,63 years 08 months,860000


## 2. Data wrangling steps
1. Reindexed dataframe using _id (unique to every resale transaction)
2. Changed room types into float values, with Executive as 5.5 rooms (extra study/balcony/bathroom)
3. Storey range was converted to avg_storey, the avg floor would be used (every value is a difference of 3 storeys)
4. Resale_price, Floor area converted to float values
5. Month was converted into datetime format, to be used to detrend the time series moving average
6. Year/Month was separated into Year and Month for visualisation purposes
7. Remaining lease was converted into remaining months (float)
8. Update capitalisation and street naming conventions (for purpose of API call later)
9. Categorised towns into regions (North, West, East, North-East, Central) https://www.hdb.gov.sg/about-us/history/hdb-towns-your-home

In [5]:
# Data transformation and geolocationing
etl_logger.info('Cleaning data')
df = clean_df(df)
etl_logger.info('\t\tCompleted')
display(df.dtypes)
df

resale_price       float64
year                 int32
month                int32
year_month          object
region              object
town                object
rooms              float64
avg_storey         float64
floor_area_sqm     float64
remaining_lease    float64
address             object
dtype: object

,resale_price,year,month,year_month,region,town,rooms,avg_storey,floor_area_sqm,remaining_lease,address
_id,,,,,,,,,,,
143397,267000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,44.0,55.416667,"406, Ang Mo Kio Avenue 10"
143398,300000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,49.0,53.500000,"323, Ang Mo Kio Avenue 3"
143399,280000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3"
143400,282000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,8.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3"
143401,289800.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,45.0,62.083333,"170, Ang Mo Kio Avenue 4"
...,...,...,...,...,...,...,...,...,...,...,...
171109,650000.0,2024,1,2024-01,North,Yishun,5.0,5.0,121.0,63.666667,"820, Yishun Street 81"
171110,850000.0,2024,1,2024-01,North,Yishun,5.5,2.0,146.0,63.416667,"391, Yishun Avenue 6"
171111,783000.0,2024,1,2024-01,North,Yishun,5.5,5.0,146.0,63.916667,"325, Yishun Ctrl"


## 3. Feature Engineering (Geodata)

Lastly, location plays a huge role in house pricing, hence

3.1 Obtaining latitude, longitude, postal codes

3.2 Distance to city center

3.3 Obtaining MRT locations

3.4 Determine nearest MRT and traveling time

### 3.1 Latitude & longitude from address
Using street name and block, I utilized OneMap API to obtain the latitude, longitude, and postal codes of each flat https://www.onemap.gov.sg/docs

In [6]:
etl_logger.info('Getting geolocations')
geo_data_df= get_location_data(df[['address']], verbose=1, cached_session=session)
display(geo_data_df.dtypes)
etl_logger.info('\t\tCompleted')
geo_data_df

lat_long        object
postal_code     object
latitude       float64
longitude      float64
numpy_array     object
dtype: object

,lat_long,postal_code,latitude,longitude,numpy_array
_id,,,,,
143397,"1.36200453938712,103.853879910407",560406,1.362005,103.853880,"[1.36200453938712, 103.853879910407]"
143398,"1.36789004826246,103.847599140207",560323,1.367890,103.847599,"[1.36789004826246, 103.847599140207]"
143399,"1.36622707120636,103.850085858983",560314,1.366227,103.850086,"[1.36622707120636, 103.850085858983]"
143400,"1.36622707120636,103.850085858983",560314,1.366227,103.850086,"[1.36622707120636, 103.850085858983]"
143401,"1.37400071781295,103.83643153142",560170,1.374001,103.836432,"[1.37400071781295, 103.83643153142]"
...,...,...,...,...,...
171109,"1.41309901104205,103.833867702757",760820,1.413099,103.833868,"[1.41309901104205, 103.833867702757]"
171110,"1.42946769237362,103.849171432797",760391,1.429468,103.849171,"[1.42946769237362, 103.849171432797]"
171111,"1.42923856240311,103.842145690691",760325,1.429239,103.842146,"[1.42923856240311, 103.842145690691]"


### 3.2 Distance to city center

The central district of Singapore has the highest housing prices. Property nearer to the city centre tend to have a higher price.

We will make use of this to create a new feature to test if it is significant in model building.

In [7]:
etl_logger.info('Getting distances to city center (Marina Bay)')
dist_to_marina_bay = multiple_distance_to(geo_data_df['numpy_array'], 'Marina Bay', dist_type='geodesic', verbose=1)
dist_to_marina_bay = pd.Series(dist_to_marina_bay, name='dist_to_marina_bay')
etl_logger.info('\t\tCompleted')

etl_logger.info('Combining geolocation data to main')
df = pd.concat([df, dist_to_marina_bay, geo_data_df['latitude'], geo_data_df['longitude'], geo_data_df['postal_code']], axis=1)
etl_logger.info('\t\tCompleted')
df

,resale_price,year,month,year_month,region,town,rooms,avg_storey,floor_area_sqm,remaining_lease,address,dist_to_marina_bay,latitude,longitude,postal_code
_id,,,,,,,,,,,,,,,
143397,267000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,44.0,55.416667,"406, Ang Mo Kio Avenue 10",8.69,1.362005,103.853880,560406
143398,300000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,49.0,53.500000,"323, Ang Mo Kio Avenue 3",9.43,1.367890,103.847599,560323
143399,280000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3",9.21,1.366227,103.850086,560314
143400,282000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,8.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3",9.21,1.366227,103.850086,560314
143401,289800.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,45.0,62.083333,"170, Ang Mo Kio Avenue 4",10.35,1.374001,103.836432,560170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171109,650000.0,2024,1,2024-01,North,Yishun,5.0,5.0,121.0,63.666667,"820, Yishun Street 81",14.62,1.413099,103.833868,760820
171110,850000.0,2024,1,2024-01,North,Yishun,5.5,2.0,146.0,63.416667,"391, Yishun Avenue 6",16.17,1.429468,103.849171,760391
171111,783000.0,2024,1,2024-01,North,Yishun,5.5,5.0,146.0,63.916667,"325, Yishun Ctrl",16.23,1.429239,103.842146,760325


### 3.3 MRT Locations
The location of all MRT stations was also obtained using OneMap API and saved as a json file locally

Load Json file and convert to numpy array to utilize matrix operations.

In [9]:
# Convert coordinates into numpy arrays
mrt_coordinates_dict = load_mrt_coordinates('static/mrt_dict.json')
mrt_stations = np.array(list(mrt_coordinates_dict.keys()))
mrt_coordinates = np.array(list(mrt_coordinates_dict.values()))

### 3.4 Nearest MRT stations and Minimum distance/time
* Using the matrix operations, we are able to find the nearest MRT station by absolute distance 
* Then use OneMap's route_api_call() to get distance/time to MRT stations

In [19]:
n_nearest_stations = 1
# Matrix operations to find nearest MRT stations for each row
etl_logger.info(f'Finding nearest stations: n={n_nearest_stations}')
nearest_stations = geo_data_df.apply(find_nearest_stations, mrt_stations= mrt_stations, mrt_coordinates=mrt_coordinates, n_nearest_stations=n_nearest_stations, axis=1, verbose=0)
nearest_stations_df = pd.DataFrame(nearest_stations.tolist(), index=geo_data_df.index, columns=['nearest_station_'+ str(x) for x in range(n_nearest_stations)] + ['dist_to_station_'+ str(x) for x in range(n_nearest_stations)])
nearest_stations_df

,nearest_station_0,dist_to_station_0
_id,,
143397,Ang Mo Kio MRT,1.00
143398,Ang Mo Kio MRT,0.31
143399,Ang Mo Kio MRT,0.41
143400,Ang Mo Kio MRT,0.41
143401,Mayflower MRT,0.28
...,...,...
171109,Yishun MRT,1.81
171110,Yishun MRT,1.58
171111,Yishun MRT,0.79


In [20]:
df = pd.concat([df, nearest_stations_df], axis=1)
etl_logger.info('\t\tCompleted')
display(df.dtypes)
df

resale_price          float64
year                    int32
month                   int32
year_month             object
region                 object
town                   object
rooms                 float64
avg_storey            float64
floor_area_sqm        float64
remaining_lease       float64
address                object
dist_to_marina_bay    float64
latitude              float64
longitude             float64
postal_code            object
nearest_station_0      object
dist_to_station_0     float64
dtype: object

,resale_price,year,month,year_month,region,town,rooms,avg_storey,floor_area_sqm,remaining_lease,address,dist_to_marina_bay,latitude,longitude,postal_code,nearest_station_0,dist_to_station_0
_id,,,,,,,,,,,,,,,,,
143397,267000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,44.0,55.416667,"406, Ang Mo Kio Avenue 10",8.69,1.362005,103.853880,560406,Ang Mo Kio MRT,1.00
143398,300000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,49.0,53.500000,"323, Ang Mo Kio Avenue 3",9.43,1.367890,103.847599,560323,Ang Mo Kio MRT,0.31
143399,280000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,5.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3",9.21,1.366227,103.850086,560314,Ang Mo Kio MRT,0.41
143400,282000.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,8.0,44.0,54.083333,"314, Ang Mo Kio Avenue 3",9.21,1.366227,103.850086,560314,Ang Mo Kio MRT,0.41
143401,289800.0,2023,1,2023-01,North-East,Ang Mo Kio,2.0,2.0,45.0,62.083333,"170, Ang Mo Kio Avenue 4",10.35,1.374001,103.836432,560170,Mayflower MRT,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171109,650000.0,2024,1,2024-01,North,Yishun,5.0,5.0,121.0,63.666667,"820, Yishun Street 81",14.62,1.413099,103.833868,760820,Yishun MRT,1.81
171110,850000.0,2024,1,2024-01,North,Yishun,5.5,2.0,146.0,63.416667,"391, Yishun Avenue 6",16.17,1.429468,103.849171,760391,Yishun MRT,1.58
171111,783000.0,2024,1,2024-01,North,Yishun,5.5,5.0,146.0,63.916667,"325, Yishun Ctrl",16.23,1.429239,103.842146,760325,Yishun MRT,0.79


In [24]:
etl_logger.info('Splitting data')
year_month = sorted(df['year_month'].unique())
etl_logger.info('\t\tTime range found:')
etl_logger.info(year_month)

In [23]:
# Save data
csv_file = f'static/from_{year_month[0]}_to_{year_month[-1]}.csv'
df.to_csv(csv_file)
etl_logger.info(f'\t\tFull data saved as "{csv_file}" @ {datetime.now()}')

# Split out most recent month as Test data, the rest as training data
test = df[df['year_month']==year_month[-1]] 
train = df[df['year_month']!=year_month[-1]] 

train.to_csv(output_file_train)
etl_logger.info(f'\t\tTraining data saved as {output_file_train} @ {datetime.now()}')
test.to_csv(output_file_test)
etl_logger.info(f'\t\tTest data saved as {output_file_test} @ {datetime.now()}')